## SQL queries example

This file is for the reason to demonstrate SQL skills

**SQL portfolio is also available at** [CodeWars solutions](https://www.codewars.com/users/yurywallet/completed_solutions)


In [2]:
from IPython.display import display, HTML

1. Get data

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  2 21:34:41 2022

@author: Yury
"""

import pandas as pd
path=r'C:\a_job\2022\smalls (cat food)\task\data'

df_o=pd.read_excel(f"{path}\Analyst Exercise - FACT_ORDERS.xlsx")
df_c=pd.read_excel(f"{path}\Analyst Exercise - DIM_CUSTOMERS.xlsx")

2. Create database tables

In [4]:
import sqlite3
# connection object
connection_obj = sqlite3.connect('smalls_database_jp.db')
 
# cursor object
cursor_obj = connection_obj.cursor()

cursor_obj.execute('''
          CREATE TABLE IF NOT EXISTS FACT_ORDERS
          ([ORDER_ID] INTEGER PRIMARY KEY, 
          [CUSTOMER_ID] INTEGER, 
          [ORDER_DATE] , 
          [ORDER_SEQUENCE] INTEGER, 
          [ORDER_TYPE] TEXT,
          [REVENUE] REAL, 
          [MARGIN] REAL
          )
          ''')
          
cursor_obj.execute('''
          CREATE TABLE IF NOT EXISTS DIM_CUSTOMERS
          ([CUSTOMER_ID] INTEGER PRIMARY KEY,
          [TRIAL_WEEK], 
          [TRIAL_PLAN] TEXT,
          [GA_SOURCE] TEXT,
          [GA_MEDIUM] TEXT,
          [CANCELLATION_FLAG] TEXT,
          [CANCELLATION_DATE], 
          [CANCELLATION_REASON] TEXT,
          [CAT_COUNT] INTEGER
          )
          ''')

connection_obj.commit()

3. Data appeared to have not unique CUSTOMER_ID in DIM_CUSTOMERS -> remove duplicates

In [5]:
tmp=df_c.groupby(['CUSTOMER_ID']).size().reset_index().copy()
tmp.rename(columns={0:"count"}, inplace =True)  
duplicate_id=tmp.loc[tmp["count"]>1]['CUSTOMER_ID'].tolist()
print("Duplicates in Customer", duplicate_id)
display(df_c.loc[df_c['CUSTOMER_ID'].isin(duplicate_id)])
df_c=df_c.loc[~(df_c['CUSTOMER_ID'].isin(duplicate_id))].copy()
df_o=df_o.loc[~(df_o['CUSTOMER_ID'].isin(duplicate_id))].copy()

Duplicates in Customer [5359139848291]


,CUSTOMER_ID,TRIAL_WEEK,TRIAL_PLAN,GA_SOURCE,GA_MEDIUM,CANCELLATION_FLAG,CANCELLATION_DATE,CANCELLATION_REASON,CAT_COUNT
35982,5359139848291,2021-08-02,FRESH,NaN,NaN,INACTIVE,2021-08-31 12:53:17,Other,1.0
36089,5359139848291,2021-08-02,FRESH,NaN,NaN,INACTIVE,2021-08-05 09:25:37,Other,1.0


4. Insert data

In [6]:
#insert
df=df_c[['CUSTOMER_ID', 'TRIAL_WEEK', 'TRIAL_PLAN', 'GA_SOURCE', 'GA_MEDIUM',
       'CANCELLATION_FLAG', 'CANCELLATION_DATE', 'CANCELLATION_REASON',
       'CAT_COUNT']].copy()
df.to_sql(name='DIM_CUSTOMERS', con=connection_obj, if_exists='append', index=False)

df=df_o[['ORDER_ID', 'CUSTOMER_ID', 'ORDER_DATE', 'ORDER_SEQUENCE', 'ORDER_TYPE',
       'REVENUE', 'MARGIN']].copy()
df.to_sql(name='FACT_ORDERS', con=connection_obj, if_exists='append', index=False)

connection_obj.commit()

5. Check data

In [7]:
#check data
cursor_obj.execute('''select * from DIM_CUSTOMERS limit 10''')
df = pd.DataFrame(cursor_obj.fetchall())    
display(df)

,0,1,2,3,4,5,6,7,8
0,7191852755,2020-06-08 00:00:00,MIX,None,None,INACTIVE,2021-06-01 15:27:36,Cat Had Reaction To Food,1.0
1,7275683667,2021-07-12 00:00:00,FRESH,affiliate,impact,ACTIVE,2022-04-20 07:52:54,None,1.0
2,7276909843,2021-12-13 00:00:00,FRESH,(direct),(none),ACTIVE,2022-03-28 20:02:16,None,3.0
3,7287750995,2022-03-14 00:00:00,FRESH,AddShoppers,safeopt,INACTIVE,2022-04-09 14:09:44,My cat won't eat,1.0
4,7328144211,2020-03-30 00:00:00,MIX,None,None,INACTIVE,2020-12-05 19:57:36,CWE - Made Effort,5.0
5,7366195155,2021-08-23 00:00:00,FRESH,facebook,retargeting,INACTIVE,2021-12-22 09:52:09,Unknown,1.0
6,7373808787,2021-03-29 00:00:00,None,None,None,INACTIVE,2021-05-05 18:53:58,Just Wanted Trial/Doesn't Want Subscription,NaN
7,7374000403,2021-09-06 00:00:00,FRESH,facebook,retargeting,INACTIVE,2021-11-01 19:57:32,CWE - Abandon Trial,1.0
8,7377522003,2021-04-19 00:00:00,FRESH,google,organic,INACTIVE,2021-05-14 17:59:59,CWE - Abandon Trial,2.0
9,8884813843,2021-04-12 00:00:00,FRESH,(direct),(none),INACTIVE,2021-05-10 18:56:50,CWE - Abandon Trial,2.0


In [8]:
cursor_obj.execute('''select * from FACT_ORDERS limit 10''')
df = pd.DataFrame(cursor_obj.fetchall()) 
display(df)

,0,1,2,3,4,5,6
0,526328430708,649548398708,2018-07-11 12:03:30,None,CANCELLED,89.50,-40.28
1,580140204131,649548398708,2018-09-20 03:15:41,None,CANCELLED,40.50,-27.62
2,628431454307,779229626467,2018-11-08 14:24:41,None,CANCELLED,27.00,-31.25
3,694858186851,842632888419,2019-01-07 18:39:17,None,CANCELLED,108.00,-60.94
4,772356505699,936707883107,2019-04-02 19:56:28,None,CANCELLED,64.00,-51.35
5,1732647387235,2461008953443,2019-10-10 10:09:11,None,CANCELLED,51.00,-48.80
6,1836155732067,2653769039971,2019-10-27 23:14:23,None,CANCELLED,25.50,-35.30
7,1892933632099,2769517838435,2019-11-23 08:13:35,None,CANCELLED,58.50,-59.46
8,1963283873891,2851777970275,2019-12-16 03:26:03,None,CANCELLED,45.75,-39.47
9,1969217241187,1841373085795,2019-12-18 09:35:27,None,CANCELLED,25.50,-42.24


6. Example of Query:
    
    **Get information about retention as of number of sequential orders for users that where acuired during month (trial_week aggregated to month)**
    
    
    

In [9]:
# get query
cursor_obj.execute('''
        WITH one as (
            SELECT *,
            --DATE_FORMAT(TRIAL_WEEK, '%Y-%m-01') as TRIAL_MONTH
            strftime('%Y-%m-01', TRIAL_WEEK) as TRIAL_MONTH
            FROM FACT_ORDERS as o
            INNER JOIN DIM_CUSTOMERS as c on c.CUSTOMER_ID=o.CUSTOMER_ID
            WHERE ORDER_TYPE='SUBSCRIPTION'
            ),
            two as (
            SELECT TRIAL_MONTH, ORDER_SEQUENCE, count(ORDER_ID) as num_orders
            FROM one
            GROUP BY 1,2
            ),
            three as (
            SELECT t.*, tt.num_orders prev_num_orders
            FROM two t
            LEFT JOIN two tt ON (t.TRIAL_MONTH=tt.TRIAL_MONTH AND t.ORDER_SEQUENCE=tt.ORDER_SEQUENCE+1)
            )
            
        SELECT *,  100.*num_orders/prev_num_orders retention
        FROM three
        ORDER BY 1        

''')

df = pd.DataFrame(cursor_obj.fetchall()
                  , columns=['TRIAL_MONTH','ORDER_SEQUENCE', 'num_orders','prev_num_orders', 'retention'])

display(df)

,TRIAL_MONTH,ORDER_SEQUENCE,num_orders,prev_num_orders,retention
0,2020-01-01,2,652,NaN,NaN
1,2020-01-01,3,475,652.0,72.852761
2,2020-01-01,4,400,475.0,84.210526
3,2020-01-01,5,350,400.0,87.500000
4,2020-01-01,6,298,350.0,85.142857
...,...,...,...,...,...
697,2022-03-01,5,3,23.0,13.043478
698,2022-04-01,2,513,NaN,NaN
699,2022-04-01,3,20,513.0,3.898635
700,2022-04-01,4,2,20.0,10.000000


7. Close connection

In [10]:
# Close the connection
connection_obj.close()